<a href="https://colab.research.google.com/github/dejung71020/YOLO/blob/main/YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path

root = Path('./RSD_datasets')
Path('./RSD_datasets/trainval').mkdir(parents=True, exist_ok=True)
Path('./RSD_datasets/test').mkdir(parents=True, exist_ok=True)

In [ ]:
for path1 in ('images', 'labels'):
    for path2 in ('train', 'val', 'test'):
        new_path = root / 'RSD' / path1 / path2
        print(new_path)
        new_path.mkdir(parents=True, exist_ok=True)

RSD_datasets/RSD/images/train
RSD_datasets/RSD/images/val
RSD_datasets/RSD/images/test
RSD_datasets/RSD/labels/train
RSD_datasets/RSD/labels/val
RSD_datasets/RSD/labels/test


In [ ]:
%cd /content/

/content


In [ ]:
!unzip -qq //content/drive/MyDrive/data/RoadSignDetection.zip

In [ ]:
# 1. 라벨 변환 도구 다운로드 (XML -> YOLO txt 변환용)
!git clone https://github.com/ssaru/convert2Yolo.git

# 2. 필요한 라이브러리 임포트
import os
import glob
import shutil
import yaml
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

# 3. 클래스 이름 추출 (road.names 생성)
# 데이터가 있는 원본 경로 (사용자 환경에 맞게 수정 가능)
ORIGIN_IMG_PATH = '/content/images'
ORIGIN_XML_PATH = '/content/annotations'

xml_files = glob.glob(os.path.join(ORIGIN_XML_PATH, '*.xml'))
classes = set()

# 모든 XML을 열어서 클래스 이름 확인
for xml_file in xml_files:
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for obj in root.findall('object'):
        classes.add(obj.find('name').text)

# 정렬된 클래스 리스트 생성 및 저장
class_list = sorted(list(classes))
print("감지된 클래스 목록:", class_list)

with open('road.names', 'w') as f:
    for cls in class_list:
        f.write(cls + '\n')

Cloning into 'convert2Yolo'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 215 (delta 38), reused 35 (delta 35), pack-reused 172 (from 1)
Receiving objects: 100% (215/215), 994.67 KiB | 26.88 MiB/s, done.
Resolving deltas: 100% (95/95), done.
감지된 클래스 목록: ['crosswalk', 'speedlimit', 'stop', 'trafficlight']


In [ ]:
# 1. 저장할 경로 설정
BASE_DIR = '/content/RSD_datasets/RSD'
TEMP_XML_DIR = '/content/temp_xml'  # XML을 변환하기 위해 잠시 모아둘 곳

# 폴더 생성
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(BASE_DIR, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(BASE_DIR, 'labels', split), exist_ok=True)
    os.makedirs(os.path.join(TEMP_XML_DIR, split), exist_ok=True)

# 2. 파일 리스트 가져오기 (이미지 파일 기준)
image_files = glob.glob(os.path.join(ORIGIN_IMG_PATH, '*.png')) # png가 아니라면 *.jpg로 변경
data_list = []

for img_path in image_files:
    filename = os.path.basename(img_path)
    xml_filename = os.path.splitext(filename)[0] + '.xml'
    xml_path = os.path.join(ORIGIN_XML_PATH, xml_filename)

    # 이미지와 XML이 짝이 맞는 경우만 추가
    if os.path.exists(xml_path):
        data_list.append((img_path, xml_path))

# 3. 데이터 분할 (Train: 80%, Val: 10%, Test: 10%)
train_data, test_data = train_test_split(data_list, test_size=0.4, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

print(f"데이터 개수 - Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}")

# 4. 파일 이동 함수
def move_files(data, split):
    for src_img, src_xml in data:
        filename = os.path.basename(src_img)
        xml_filename = os.path.basename(src_xml)

        # 이미지 이동 -> images 폴더로
        shutil.copy(src_img, os.path.join(BASE_DIR, 'images', split, filename))

        # XML 이동 -> 임시 xml 폴더로 (나중에 변환)
        shutil.copy(src_xml, os.path.join(TEMP_XML_DIR, split, xml_filename))

# 실제로 이동 실행
move_files(train_data, 'train')
move_files(val_data, 'val')
move_files(test_data, 'test')

print("파일 이동 완료!")

데이터 개수 - Train: 526, Val: 175, Test: 176
파일 이동 완료!


In [ ]:
# 1. convert2Yolo 폴더로 이동 및 설치
%cd /content/convert2Yolo
!pip install -qr requirements.txt

# 2. 변환 실행 (Train, Val, Test 각각)
splits = ['train', 'val', 'test']

for split in splits:
    print(f"\nConverting {split} data...")
    # 아래 명령어는 XML을 읽어서 YOLO 형식 TXT로 변환하여 labels 폴더에 저장합니다.
    !python example.py \
        --datasets VOC \
        --img_path /content/RSD_datasets/RSD/images/{split} \
        --label /content/temp_xml/{split} \
        --convert_output_path /content/RSD_datasets/RSD/labels/{split} \
        --img_type '.png' \
        --manifest_path /content/ \
        --cls_list_file /content/road.names

# 다시 메인 폴더로 복귀
%cd /content

/content/convert2Yolo
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 12.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.

Converting train data...

VOC Parsing:   |████████████████████████████████████████| 100.0% (701/701)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (701/701)  Complete


YOLO Saving:   |███████████████████████████████

In [ ]:
# 1. convert2Yolo 폴더로 이동 및 설치
%cd /content/convert2Yolo
!pip install -qr requirements.txt

# 2. 변환 실행 (Train, Val, Test 각각)
splits = ['train', 'val', 'test']

for split in splits:
    print(f"\nConverting {split} data...")
    # 아래 명령어는 XML을 읽어서 YOLO 형식 TXT로 변환하여 labels 폴더에 저장합니다.
    !python example.py \
        --datasets VOC \
        --img_path /content/RSD_datasets/RSD/images/{split} \
        --label /content/temp_xml/{split} \
        --convert_output_path /content/RSD_datasets/RSD/labels/{split} \
        --img_type '.png' \
        --manifest_path /content/ \
        --cls_list_file /content/road.names

# 다시 메인 폴더로 복귀
%cd /content

/content/convert2Yolo
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.

Converting train data...

VOC Parsing:   |████████████████████████████████████████| 100.0% (701/701)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (701/701)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (701/701)  Complete


Converting val data...

VOC Parsing:   |████████████████████████████████████████| 100.0% (209/209)  Complete


YOLO Genera

In [ ]:
# 1. data.yaml 파일 생성
data = {
    'train': '/content/RSD_datasets/RSD/images/train',
    'val':   '/content/RSD_datasets/RSD/images/val',
    'test':  '/content/RSD_datasets/RSD/images/test',
    'nc': len(class_list),
    'names': class_list
}

with open('/content/RSD_datasets/data.yaml', 'w') as f:
    yaml.dump(data, f)

print("data.yaml 생성 완료:", data)


# 2. YOLOv8 설치 및 학습
!pip install ultralytics

from ultralytics import YOLO

# 모델 로드 (yolov8s.pt 사용)
model = YOLO('yolov8s.pt')

# 학습 시작
model.train(
    data='/content/RSD_datasets/data.yaml',
    epochs=30,      # 학습 횟수
    imgsz=640,      # 이미지 크기
    batch=16,       # 배치 사이즈
    name='road_sign_detection' # 결과 저장 폴더 이름
)

data.yaml 생성 완료: {'train': '/content/RSD_datasets/RSD/images/train', 'val': '/content/RSD_datasets/RSD/images/val', 'test': '/content/RSD_datasets/RSD/images/test', 'nc': 4, 'names': ['crosswalk', 'speedlimit', 'stop', 'trafficlight']}
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/RSD_dataset

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f3a58659100>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0